# Sandstone 2D parallel-beam data reconstruction demo



In [ ]:
# Import all CIL components needed
from cil.framework import ImageData, ImageGeometry
from cil.framework import AcquisitionGeometry, AcquisitionData

# CIL Processors
from cil.processors import CentreOfRotationCorrector, Slicer, TransmissionAbsorptionConverter, Normaliser #, Padder

from Padder import Padder

from cil.utilities.display import show2D, show_geometry

# Import from cil.plugins.astra
from cil.plugins.astra.processors import FBP
from cil.plugins.astra.operators import ProjectionOperator

# All external imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

In [ ]:
import logging
logging.basicConfig(level = logging.INFO)

Load demo data set and display the first raw projection

In [ ]:
#all_data = scipy.io.loadmat("/media/newhd/shared/Data/synchrotron/small/slice_0270_data.mat")
#all_data = scipy.io.loadmat("/media/newhd/shared/Data/synchrotron/small/slice_0540_data.mat")
all_data = scipy.io.loadmat("/media/newhd/shared/Data/synchrotron/small/slice_0810_data.mat")
#all_data = scipy.io.loadmat("/media/newhd/shared/Data/synchrotron/small/slice_1080_data.mat")

In [ ]:
darks = all_data['X_dark']
flats = all_data['X_flat']
projs = all_data['X_proj']
projs.shape

In [ ]:
ag = AcquisitionGeometry.create_Parallel2D()  \
         .set_panel(num_pixels=(2560))        \
         .set_angles(angles=np.linspace(0,180,1500,endpoint=False))

In [ ]:
show_geometry(ag)

In [ ]:
ag.dimension_labels

In [ ]:
data = AcquisitionData(np.transpose(projs),False,geometry=ag)

In [ ]:
show2D(data)

In [ ]:
ig = ag.get_ImageGeometry()

In [ ]:
rec1 = FBP(ig,ag)(data)

In [ ]:
show2D(rec1)

In [ ]:
data2 = Normaliser(flat_field=flats.mean(axis=1),
                   dark_field=darks.mean(axis=1)
                  )(data)

In [ ]:
show2D(data2)

In [ ]:
rec2 = FBP(ig,ag)(data2)

In [ ]:
show2D(rec2)

Pad on both sides for region-of-interest correction

In [ ]:
data3 = TransmissionAbsorptionConverter()(data2)

In [ ]:
rec3 = FBP(ig,ag)(data3)

In [ ]:
show2D(rec3)

In [ ]:
padsize = 600
data4 = Padder.edge(pad_width={'horizontal': padsize})(data3)

In [ ]:
show2D(data4)

In [ ]:
rec4 = FBP(ig,data4.geometry)(data4)

In [ ]:
show2D(rec4)

In [ ]:
data5 = CentreOfRotationCorrector.image_sharpness(FBP=FBP,search_range=100, tolerance=0.1)(data4)

In [ ]:
rec5 = FBP(ig,data5.geometry,device='gpu')(data5)

In [ ]:
show2D(rec5)

Load and plot the reconstruction provided from the synchrotron (rotate to match our orientation)

In [ ]:
#vendor_recon = plt.imread("/media/newhd/shared/Data/synchrotron/small/recon/BBii_0270.rec.16bit.tif")
#vendor_recon = plt.imread("/media/newhd/shared/Data/synchrotron/small/recon/BBii_0540.rec.16bit.tif")
vendor_recon = plt.imread("/media/newhd/shared/Data/synchrotron/small/recon/BBii_0810.rec.16bit.tif")
#vendor_recon = plt.imread("/media/newhd/shared/Data/synchrotron/small/recon/BBii_1080.rec.16bit.tif")

In [ ]:
show2D(np.rot90(vendor_recon))